In [ ]:
from __future__ import print_function
import argparse

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from scipy.io import loadmat,savemat
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms
from dbpn import BasicNet as BDBPN
import numpy as np
from dataset import *
from data import get_eval_set
from functools import reduce

#from scipy.misc import save
import scipy.io as sio
import time
import cv2


In [ ]:
#Number of GPUs
gpus=1
# Whether to use GPU (=True) or not (=False)
gpu_mode=True
#random seed to use. Default=123
seed=123
# Super-resolution upscale factor
upscale_factor=2
# Parent dirctory where test data is stored
input_dir='Input'
# Actual test dataset directory
test_dataset='CCPs'
# Number of threads for data loader to use
threads=1
# Testing batch size
testBatchSize=1
# Type of the trained model to use
model_type='BDBPN' # BDBPN
# SR pretrained base model
# DBPNCCP2x_epoch_65.pth
model_name='models/Pretrained/DBPNCCP2x_epoch_65.pth' # Inference model for all CCPs
# Base location to save results
output='Results/'
# parser.add_argument('--chop_forward', type=bool, default=False)
chop_forward=False
# parser.add_argument('--self_ensemble', type=bool, default=False)
self_ensemble=False
# parser.add_argument('--residual', type=bool, default=False)
residual=False
# Fluorescence Level
fluores_level= 1

In [ ]:
gpus_list=range(gpus)
#print(opt)

cuda = gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

# help='random seed to use. Default=123'; opt.seed=123
torch.manual_seed(seed)
if cuda:
    #torch.cuda.manual_seed(opt.seed)
    torch.cuda.manual_seed(seed)

### Loading the Test dataset

In [ ]:
print('===> Loading datasets')
root='BioSR/Testing/CCPs/test_wf/'
data_transform = transforms.Compose([transforms.ToTensor()])

# Create the dataset
# test_set = get_eval_set(os.path.join(opt.input_dir,opt.test_dataset), opt.upscale_factor)
test_set = ImagePairTestDataset(root,upscale_factor=2, flevel=fluores_level,norm_flag=1)

# Create the DataLoader
test_data_loader = DataLoader(test_set, testBatchSize, shuffle=True)

In [ ]:
print('===> Building model')


if model_type == 'BDBPN':
    model = BDBPN(num_channels=1, base_filter=64,  feat = 256, num_stages=7, scale_factor=upscale_factor) ###D-DBPN
    
if cuda:
    model = torch.nn.DataParallel(model, device_ids=gpus_list)

chk= torch.load(model_name, map_location=lambda storage, loc: storage)
model.load_state_dict(chk['model_state_dict'],strict=False)
#optimizer.load_state_dict(chk['optimizer_state_dict'])
#model.load_state_dict(torch.load(model_name, map_location=lambda storage, loc: storage))
print('Pre-trained SR model is loaded.')

if cuda:
    model = model.cuda(gpus_list[0])

In [ ]:
import tifffile
import cv2
def save_img(img, img_name):
    #save_img = img.squeeze().clamp(0, 1).numpy().transpose(1,2,0)
    save_img = img.squeeze().numpy()
    #save_img = img.squeeze().clamp(0,1).numpy()
    # save img
    save_dir=os.path.join(output,test_dataset+"/level_0{}".format(fluores_level))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    #name,ext=os.path.splitext(img_name) # extra code to store as .mat file
    save_fn = save_dir +'/'+ img_name # code to store as img file
    #save_fn = save_dir +'/'+ name+'.mat'  # Code to store as .mat file
    print(save_fn)
    #result = Image.fromarray((save_img * 255).astype(np.uint8))
    #result.save(save_dir +'/'+ img_name)
    #savemat(save_dir+'/'+img_name+'.mat',{img_name: save_img})
    #cv2.imwrite(save_fn, cv2.cvtColor(save_img*255, cv2.COLOR_BGR2RGB),  [cv2.IMWRITE_PNG_COMPRESSION, 0])
    #cv2.imwrite(save_fn, cv2.cvtColor(save_img, cv2.COLOR_BGR2GRAY))
    #tifffile.imwrite(save_fn,np.uint16(save_img*65535)) # code to store as img file
    tifffile.imwrite(save_fn,np.uint8(save_img*300)) # Inference model for all CCPs, # code to store as img file
    
    #savemat(save_fn,{'prop':save_img}) # # Code to store as .mat file
    #tifffile.imwrite(save_fn,save_img*65535)
    #tifffile.imwrite(save_fn,save_img)
    #cv2.imwrite(save_fn,save_img)
    #cv2.imwrite(save_fn, cv2.cvtColor(save_img*255, cv2.COLOR_BGR2RGB))

In [ ]:
def eval():
    model.eval()
    for batch in test_data_loader:
        with torch.no_grad():
            input, bicubic, name = Variable(batch[0]), Variable(batch[1]), batch[2]
        if cuda:
            input = input.cuda(gpus_list[0])
            bicubic = bicubic.cuda(gpus_list[0])

        t0 = time.time()
        if chop_forward:
            with torch.no_grad():
                prediction = chop_forward(input, model, upscale_factor)
        else:
            if self_ensemble:
                with torch.no_grad():
                    prediction = x8_forward(input, model)
            else:
                with torch.no_grad():
                    #prediction = model(input.unsqueeze(1))
                    prediction = model(input)
                    #prediction = percentile_normalize(model(input))
                    
                
        if residual:
            prediction = prediction + bicubic

        t1 = time.time()
        print("===> Processing: %s || Timer: %.4f sec." % (name[0], (t1 - t0)))
        save_img(prediction.cpu().data, name[0])
        #save_img(prediction, name[0])

In [ ]:
# Eval Start!
eval()